In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/nlp_project

/content/drive/MyDrive/nlp_project


In [3]:
!pip install arabic-stopwords
!pip install qalsadi
!pip install pyarabic
!pip install gensim
!pip install top2vec
# !pip install --upgrade numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np
import pandas as pd
import spacy
import string
import qalsadi.lemmatizer
from nltk.stem.isri import ISRIStemmer
from pyarabic.araby import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
import torch
from torch import nn
from tqdm import tqdm
import pickle

from gru_model import ArabicDataset, Classifier, evaluate, train
from pre_processing_post import processPost
from feature_extraction import get_ngram_features, get_word_embedding_features, avg_word_vector, get_word_count_features

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
# needed functions
def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

# Read train data

In [6]:
train_data = pd.read_csv('./DataSet/train.csv',sep=',',header=0)
test_data = pd.read_csv('./DataSet/dev.csv',sep=',',header=0)
train_data.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,celebrity,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,info_news,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,info_news,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,celebrity,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,personal,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",info_news,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,info_news,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,personal,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,unrelated,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,info_news,1


In [7]:
test_data['category'].astype('category').cat.categories

Index(['advice', 'celebrity', 'info_news', 'others', 'personal', 'plan',
       'requests', 'restrictions', 'rumors', 'unrelated'],
      dtype='object')

In [8]:
#remove first row that has the header
train_data['category'] = train_data['category'].astype('category').cat.codes
test_data['category'] = test_data['category'].astype('category').cat.codes
train_data.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,1,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,2,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,2,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,1,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,4,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",2,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,2,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,4,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,9,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,2,1


## Over Sampling

In [9]:
# !pip install imblearn

In [10]:
# from collections import Counter
# from imblearn.over_sampling import RandomOverSampler
# train_data=train_data.drop('category',axis=1)
# y=train_data['stance']
# print(Counter(train_data['stance']))
# train_data=train_data.drop('stance',axis=1)
# # define oversampling strategy
# oversample = RandomOverSampler(random_state=3)
# # fit and apply the transform
# train_data["text"], train_data['stance'] = oversample.fit_resample(train_data, y)
# print(Counter(train_data['stance']))

# Pre-Processing the tweets

In [11]:
print(train_data["text"][9])
train_data["text"] = train_data['text'].apply(lambda x: processPost(x))
test_data['text'] = test_data['text'].apply(lambda x: processPost(x))
print(train_data["text"][9])

دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية لقاح لعد ما اتابع الاخبار هم بكل مجالاتهم متفوقين وراح يطلع اللقاح قريباً؟<LF>#دعبول_دومه_مسحول
دعبول حضر من نت طلب قائد دول إسلام قاح عد تابع اخبار مجال متفوق طلع قاح قريبا دعبول دوم مسحول


## Ara2Vec Embedding

In [12]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

In [13]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        # preprocessed = processPost(text)
        return self.tokenizer(text)

In [14]:
# Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)

## create vocablary

In [15]:
train_data_tokenized = train_data['text'].apply(tokenize)
test_data_tokenized = test_data['text'].apply(tokenize)
#merge all the sentences in one list
vocab = [item for sublist in train_data_tokenized for item in sublist]
vocab = list(set(vocab))
vocab.append('<فراغ>')
vocab.insert(0, '<مجهول>')
word2index = {word: i for i, word in enumerate(vocab)}

In [16]:
# save the word2index dictionary
with open('./vocab/category/word2index.pickle', 'wb') as handle:
    pickle.dump(word2index, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Feature Extraction

## 1. TD-IDF

In [17]:
ngramdata_features, word_vectorizer = get_ngram_features(train_data)
ngramdata_features.head()

# save the model to disk
filename = './models/category/TFIDFVectorizer.sav'
pickle.dump(word_vectorizer, open(filename, 'wb'))

LinearSVC Classifier

In [18]:
clf = SVC(kernel='linear',class_weight="balanced", probability=True)
pipe_tfidf = make_pipeline(word_vectorizer, clf)
pipe_tfidf.fit(train_data['text'], train_data['category'])
# y_pred = pipe_tfidf.predict(test_data['text'])
print_report(pipe_tfidf, test_data['text'], test_data['category'])

# save the model to disk
filename = './models/category/LinearSVC_tfidf.sav'
pickle.dump(pipe_tfidf, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       0.10      0.40      0.16        10
           1       0.80      0.82      0.81       145
           2       0.80      0.38      0.52       545
           3       0.04      0.18      0.07        17
           4       0.53      0.44      0.48       128
           5       0.24      0.49      0.32        82
           6       0.12      0.30      0.18        20
           7       0.25      1.00      0.40         2
           8       0.04      0.13      0.06        15
           9       0.18      0.44      0.25        36

    accuracy                           0.46      1000
   macro avg       0.31      0.46      0.32      1000
weighted avg       0.65      0.46      0.50      1000

accuracy: 0.457


RandomForestClassifier

In [19]:
X_train_tfidf = word_vectorizer.fit_transform(train_data['text'])
X_test_tfidf = word_vectorizer.transform(test_data['text'])
rf = RandomForestClassifier(class_weight="balanced")
rf_tfidf = rf.fit(X_train_tfidf, train_data['category'])
y_pred = rf_tfidf.predict(X_test_tfidf)

print_report(rf_tfidf, X_test_tfidf, test_data['category'])

# save the model to disk
filename = './models/category/RandomForest_tfidf.sav'
pickle.dump(rf_tfidf, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       1.00      0.10      0.18        10
           1       0.84      0.74      0.79       145
           2       0.66      0.87      0.75       545
           3       0.00      0.00      0.00        17
           4       0.54      0.35      0.43       128
           5       0.18      0.04      0.06        82
           6       0.23      0.15      0.18        20
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00        15
           9       0.48      0.42      0.45        36

    accuracy                           0.65      1000
   macro avg       0.39      0.27      0.28      1000
weighted avg       0.60      0.65      0.61      1000

accuracy: 0.650


naive bayes

In [20]:
X_train_tfidf = word_vectorizer.fit_transform(train_data['text'])
X_test_tfidf = word_vectorizer.transform(test_data['text'])
CNB_clr = ComplementNB()
CNB_clr_tfidf = CNB_clr.fit(X_train_tfidf, train_data['category'])
print_report(CNB_clr_tfidf, X_test_tfidf, test_data['category'])

# save the model to disk
filename = './models/category/NaiveBayes_tfidf.sav'
pickle.dump(CNB_clr, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.66      0.84      0.74       145
           2       0.75      0.75      0.75       545
           3       0.00      0.00      0.00        17
           4       0.41      0.70      0.52       128
           5       0.29      0.06      0.10        82
           6       0.20      0.15      0.17        20
           7       0.00      0.00      0.00         2
           8       0.25      0.07      0.11        15
           9       0.23      0.08      0.12        36

    accuracy                           0.63      1000
   macro avg       0.28      0.27      0.25      1000
weighted avg       0.60      0.63      0.60      1000

accuracy: 0.632


## 2.BoW

In [21]:
train_word_features, test_word_features, word_counter_vectorizer = get_word_count_features(train_data, test_data)

In [22]:
# save the model to disk
filename = './models/category/BoWVectorizer.sav'
pickle.dump(word_counter_vectorizer, open(filename, 'wb'))

LinearSVC Classifire

In [23]:
clf = SVC(kernel="linear",class_weight="balanced", probability=True) #{-1:0.66, 0:0.29, 1:0.05}
clf.fit(train_word_features, train_data['category'])
print_report(clf, test_word_features, test_data['category'])

# save the model to disk
filename = './models/category/LinearSVC_BoW.sav'
pickle.dump(clf, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       0.12      0.30      0.18        10
           1       0.76      0.83      0.79       145
           2       0.74      0.49      0.59       545
           3       0.04      0.12      0.06        17
           4       0.45      0.51      0.47       128
           5       0.22      0.38      0.27        82
           6       0.19      0.30      0.24        20
           7       0.00      0.00      0.00         2
           8       0.08      0.13      0.10        15
           9       0.27      0.44      0.34        36

    accuracy                           0.51      1000
   macro avg       0.29      0.35      0.30      1000
weighted avg       0.60      0.51      0.54      1000

accuracy: 0.513


RandomForest Classifier

In [24]:
rf = RandomForestClassifier(class_weight="balanced")
rf_Bow = rf.fit(train_word_features, train_data['category'])

print_report(rf_Bow, test_word_features, test_data['category'])

# save the model to disk
filename = './models/category/RandomForest_Bow.sav'
pickle.dump(rf_Bow, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       0.50      0.20      0.29        10
           1       0.83      0.76      0.79       145
           2       0.68      0.85      0.75       545
           3       0.00      0.00      0.00        17
           4       0.59      0.42      0.49       128
           5       0.10      0.02      0.04        82
           6       0.21      0.15      0.18        20
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00        15
           9       0.38      0.39      0.38        36

    accuracy                           0.65      1000
   macro avg       0.33      0.28      0.29      1000
weighted avg       0.60      0.65      0.61      1000

accuracy: 0.648


naive bayes

In [25]:
CNB_clr = ComplementNB()
CNB_clr_BoW = CNB_clr.fit(train_word_features, train_data['category'])
print_report(CNB_clr_BoW,test_word_features, test_data['category'])

# save the model to disk
filename = './models/category/NaiveBayes_BoW.sav'
pickle.dump(CNB_clr, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       1.00      0.10      0.18        10
           1       0.63      0.83      0.72       145
           2       0.74      0.78      0.76       545
           3       0.00      0.00      0.00        17
           4       0.43      0.67      0.53       128
           5       0.40      0.07      0.12        82
           6       0.25      0.15      0.19        20
           7       0.00      0.00      0.00         2
           8       1.00      0.07      0.12        15
           9       0.33      0.08      0.13        36

    accuracy                           0.64      1000
   macro avg       0.48      0.28      0.28      1000
weighted avg       0.63      0.64      0.61      1000

accuracy: 0.644


## 3.CBOW

In [26]:
X_train_vect_avg, X_test_vect_avg = get_word_embedding_features(train_data, test_data)

LinearSVC Classifier

In [27]:
clf = SVC(kernel='linear', class_weight="balanced", probability=True)
clf.fit(X_train_vect_avg, train_data['category'])
print_report(clf, X_test_vect_avg, test_data['category'])

# save the model to disk
filename = './models/category/LinearSVC_CBOW.sav'
pickle.dump(clf, open(filename, 'wb'))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.01      0.60      0.02        10
           1       0.16      0.20      0.18       145
           2       0.66      0.09      0.16       545
           3       0.00      0.00      0.00        17
           4       0.14      0.08      0.10       128
           5       0.00      0.00      0.00        82
           6       0.02      0.10      0.04        20
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00        36

    accuracy                           0.10      1000
   macro avg       0.10      0.11      0.05      1000
weighted avg       0.40      0.10      0.13      1000

accuracy: 0.098


RandomForestClassifier

In [28]:
rf = RandomForestClassifier(class_weight="balanced")
rf_vect = rf.fit(X_train_vect_avg, train_data['category'].values.ravel())
print_report(rf_vect, X_test_vect_avg, test_data['category'])

# save the model to disk
filename = './models/category/RandomForest_CBOW.sav'
pickle.dump(rf, open(filename, 'wb'))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.84      0.32      0.47       145
           2       0.58      0.94      0.72       545
           3       0.00      0.00      0.00        17
           4       0.11      0.02      0.03       128
           5       0.10      0.01      0.02        82
           6       0.20      0.10      0.13        20
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00        15
           9       0.67      0.17      0.27        36

    accuracy                           0.57      1000
   macro avg       0.25      0.16      0.16      1000
weighted avg       0.49      0.57      0.48      1000

accuracy: 0.572


## 4.Ara2Vec

In [29]:
train_data_embeddings = np.array([np.array([nlp(i).vector for i in ls if i in vocab]) for ls in train_data["text"]], dtype=object)
test_data_embeddings = np.array([np.array([nlp(i).vector for i in ls if i in vocab]) for ls in test_data["text"]], dtype=object)

In [30]:
X_train_vect_avg, X_test_vect_avg = avg_word_vector(train_data_embeddings, test_data_embeddings)

LinearSVC Classifier

In [31]:
clf = SVC(kernel='linear', class_weight="balanced", probability=True)
clf.fit(X_train_vect_avg, train_data['category'])
print_report(clf, X_test_vect_avg, test_data['category'])

# save the model to disk
filename = './models/category/LinearSVC_Ara2Vec.sav'
pickle.dump(clf, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       0.03      0.40      0.05        10
           1       0.49      0.45      0.47       145
           2       0.71      0.14      0.23       545
           3       0.00      0.00      0.00        17
           4       0.34      0.21      0.26       128
           5       0.12      0.22      0.15        82
           6       0.07      0.35      0.12        20
           7       0.01      0.50      0.02         2
           8       0.00      0.00      0.00        15
           9       0.10      0.31      0.15        36

    accuracy                           0.21      1000
   macro avg       0.19      0.26      0.14      1000
weighted avg       0.52      0.21      0.25      1000

accuracy: 0.207


RandomForestClassifier

In [32]:
rf = RandomForestClassifier(class_weight="balanced")
rf_vect = rf.fit(X_train_vect_avg, train_data['category'].values.ravel())
print_report(rf_vect, X_test_vect_avg, test_data['category'])

# save the model to disk
filename = './models/category/RandomForest_Ara2Vec.sav'
pickle.dump(rf, open(filename, 'wb'))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.79      0.40      0.53       145
           2       0.59      0.94      0.72       545
           3       0.00      0.00      0.00        17
           4       0.33      0.04      0.07       128
           5       0.09      0.01      0.02        82
           6       0.20      0.10      0.13        20
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00        15
           9       0.64      0.19      0.30        36

    accuracy                           0.59      1000
   macro avg       0.26      0.17      0.18      1000
weighted avg       0.51      0.59      0.50      1000

accuracy: 0.586


## create initial embedding matrix 

In [33]:
weights_train_matrix = []
for word in vocab:
  weights_train_matrix.append(nlp(word).vector)

weights_train_matrix = torch.from_numpy(np.array(weights_train_matrix))
weights_train_matrix.size()

torch.Size([12538, 100])

# ArabicDataset
The class that impelements the dataset for arabic tweets

In [86]:
model = Classifier(weights_train_matrix, n_classes=10, n_layer=3)
print(model)

Classifier(
  (embedding): Embedding(12538, 100)
  (GRU): GRU(100, 100, num_layers=3, batch_first=True)
  (linear): Linear(in_features=100, out_features=10, bias=True)
)


# Training

In [40]:
train_data_tokenized_as_num = train_data_tokenized.apply(lambda x: [word2index[word] for word in x])
# apply the same tokenization to the test set
test_data_tokenized_as_num = test_data_tokenized.apply(lambda x: [word2index[word] for word in x if word in word2index])
train_dataset = ArabicDataset(list(train_data_tokenized_as_num), train_data['category'], word2index['<فراغ>'])
test_dataset = ArabicDataset(list(test_data_tokenized_as_num), test_data['category'], word2index['<فراغ>'])

In [41]:
from collections import Counter
classes_count = Counter(train_data['category'])
AVG_NUM = 698.8
print(classes_count[2])

3616


In [90]:
train(model, train_dataset, class_weights=torch.tensor([AVG_NUM/classes_count[0], 
                                                        AVG_NUM/classes_count[1],
                                                        AVG_NUM/classes_count[2],
                                                        AVG_NUM/classes_count[3],
                                                        AVG_NUM/classes_count[4],
                                                        AVG_NUM/classes_count[5],
                                                        AVG_NUM/classes_count[6],
                                                        AVG_NUM/classes_count[7],
                                                        AVG_NUM/classes_count[8],
                                                        AVG_NUM/classes_count[9]]), epochs=1, learning_rate=0.001)

100%|██████████| 219/219 [00:52<00:00,  4.18it/s]

Epochs: 1 | Train Loss: 0.7831833802809999         | Train Accuracy: 0.4765312075614929



# Evaluation

In [91]:
prob = evaluate(model, test_dataset)

100%|██████████| 32/32 [00:00<00:00, 35.27it/s]

accuracy: 0.467
              precision    recall  f1-score   support

           0       0.06      0.20      0.09        10
           1       0.80      0.81      0.81       145
           2       0.76      0.39      0.52       545
           3       0.16      0.29      0.21        17
           4       0.35      0.59      0.44       128
           5       0.25      0.29      0.27        82
           6       0.07      0.25      0.11        20
           7       0.11      0.50      0.18         2
           8       0.12      0.07      0.09        15
           9       0.21      0.61      0.31        36

    accuracy                           0.47      1000
   macro avg       0.29      0.40      0.30      1000
weighted avg       0.61      0.47      0.50      1000


Test Accuracy: 0.46700000762939453


In [95]:
# save the model to disk
filename = './models/category/GRU_Ara2Vec.pth'
torch.save(model.state_dict(), filename)

with open('./probability/GRU_prob.pickle', 'wb') as handle:
    pickle.dump(prob, handle, protocol=pickle.HIGHEST_PROTOCOL)
